In [9]:
from scipy.stats import pearsonr
import numpy as np
import pandas as pd

df = pd.read_csv("/labmed/workspace/lotta/finaletoolkit/dataframes_for_ba/final_feature_matrix_blacklist_filtered_50000.tsv", sep="\t" )

gc_corrected_df = pd.read_csv("/labmed/workspace/lotta/finaletoolkit/dataframes_for_ba/final_feature_matrix_gc_corrected_50000.tsv", sep="\t" )
sample = df["sample"].unique()[0]
print(sample)
for col in ["median", "stdev", "wps_value", "min", "max", "mean" ]:


    before = df[df["sample"]==sample].copy()
    after  = gc_corrected_df[gc_corrected_df["sample"]==sample].copy()

    mask_b = np.isfinite(before["GC"]) & np.isfinite(before[col])
    mask_a = np.isfinite(after["GC"]) & np.isfinite(after[col])

    r_before = pearsonr(before.loc[mask_b, "GC"], before.loc[mask_b, col])[0]
    r_after  = pearsonr(after.loc[mask_a, "GC"],  after.loc[mask_a, col])[0]

    print(f"Pearson r of {col} before:", r_before)
    print(f"Pearson r of {col} after: ", r_after)


EE85727
Pearson r of median before: -0.4165029328153928
Pearson r of median after:  -0.10372123060938697
Pearson r of stdev before: -0.16546555336817728
Pearson r of stdev after:  -0.02905267805663862
Pearson r of wps_value before: 0.48057000661654703
Pearson r of wps_value after:  -0.017108983659441555
Pearson r of min before: 0.24401006154993485
Pearson r of min after:  -0.010086655121775418
Pearson r of max before: -0.3795140089588334
Pearson r of max after:  -0.1314034619074623
Pearson r of mean before: -0.3472218013342304
Pearson r of mean after:  -0.08570707492159162


In [4]:
import pandas as pd

col = "mean"
sample = df["sample"].unique()[0]

tmp_b = df[df["sample"]==sample][["GC", col]].dropna()
tmp_a = gc_corrected_df[gc_corrected_df["sample"]==sample][["GC", col]].dropna()

tmp_b["gc_bin"] = pd.qcut(tmp_b["GC"], q=20, duplicates="drop")
tmp_a["gc_bin"] = pd.qcut(tmp_a["GC"], q=20, duplicates="drop")

binned_b = tmp_b.groupby("gc_bin")[col].mean()
binned_a = tmp_a.groupby("gc_bin")[col].mean()

print("Range before:", (binned_b.max() - binned_b.min()))
print("Range after: ", (binned_a.max() - binned_a.min()))


Range before: 31.225635743604855
Range after:  10.93518379069306


In [7]:
from scipy.stats import pearsonr
import numpy as np

def mean_abs_corr(data, col):
    rs = []
    for s in data["sample"].unique():
        sub = data[data["sample"]==s][["GC", col]].dropna()
        if len(sub) < 50:
            continue
        r = pearsonr(sub["GC"], sub[col])[0]
        rs.append(abs(r))
    return np.mean(rs), np.median(rs), len(rs)

for col in ["median", "stdev", "wps_value", "min", "max", "mean"]:
    mean_b, med_b, n_b = mean_abs_corr(df, col)
    mean_a, med_a, n_a = mean_abs_corr(gc_corrected_df, col)

    print(col, "abs(corr) before: mean", mean_b, "median", med_b, "n", n_b)
    print(col, "abs(corr) after : mean", mean_a, "median", med_a, "n", n_a)


median abs(corr) before: mean 0.057208941847318895 median 0.03751678091234924 n 262
median abs(corr) after : mean 0.014166613409697458 median 0.006899002533731177 n 262
stdev abs(corr) before: mean 0.04752114730369109 median 0.029597285830128314 n 262
stdev abs(corr) after : mean 0.0075125497131507575 median 0.004396941784397835 n 262
wps_value abs(corr) before: mean 0.05472345492393974 median 0.022296698558573285 n 262
wps_value abs(corr) after : mean 0.007303423919426675 median 0.00449608307084693 n 262
min abs(corr) before: mean 0.049100254393907576 median 0.032175099446313846 n 262
min abs(corr) after : mean 0.009079081808171963 median 0.007226311520829707 n 262
max abs(corr) before: mean 0.059513454583973016 median 0.03525758708062418 n 262
max abs(corr) after : mean 0.012211267593452991 median 0.00464293744793412 n 262
mean abs(corr) before: mean 0.0565165525966777 median 0.039803297408768985 n 262
mean abs(corr) after : mean 0.0133544622147048 median 0.006096985567258354 n 262
